In [22]:
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
import warnings
from fake_useragent import UserAgent

class Webscraper():
    
    def __init__(self, urls):
        self.data = None
        self.urls = list(urls)
        self.smm_dict = None
         
    def scrap_smm_info_from_pages(self):

        social_media_patterns = ['www.facebook.com', 'www.instagram.com', 'www.linkedin.com', 'www.twitter.com',
                                  'facebook.com', 'instagram.com', 'linkedin.com', 'twitter.com', 'x.com']
        ua = UserAgent()

        fake_headers = ua.chrome
        smm_dict = {}
        # warnings.filterwarnings('ignore', category=requests.packages.urllib3.exceptions.InsecureRequestWarning)
        
        for id4, smm_find in enumerate(self.urls):
            smm_links = set()
            try:
                response2 = requests.get(smm_find, timeout=8, headers={'User-Agent' : fake_headers}, verify=False)
                soupy = BeautifulSoup(response2.text, 'html.parser')
                body = soupy.find('body')

                if body is not None:
                    href_values = [link.get('href') for link in body.find_all('a') if link.get('href')]

                    for href in href_values:
                        for pattern in social_media_patterns:
                            if pattern in href:
                                smm_links.add(href)

            except Exception as e:
                pass

            if smm_links:
                smm_dict[id4] = list(smm_links)

        self.smm_dict = smm_dict
        return smm_dict



In [24]:
urls = ['https://familyfriendsvet.com', 'https://bluepearlvet.com', 'https://www.westmichiganaeh.com', 'https://willowrunvetservices.com']

scraper = Webscraper(urls)

result = scraper.scrap_smm_info_from_pages()
result 

# Scrapper works initally, now need to get rid of warning and add print statments , then on to next logics 
# Make github push tomorrow, also youll watch vid to make lirary after oop is done

c:\Users\Nick\anaconda3\envs\myenviorment\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'familyfriendsvet.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\Nick\anaconda3\envs\myenviorment\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'bluepearlvet.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\Nick\anaconda3\envs\myenviorment\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.westmichiganaeh.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings

{0: ['https://www.facebook.com/familyfriendsvet/',
  'https://www.instagram.com/familyfriendsveterinary/'],
 1: ['https://www.facebook.com/BluePearlPetHospital/',
  'https://twitter.com/bluepearlvet',
  'https://www.instagram.com/bluepearlvet/',
  'https://www.linkedin.com/company/bluepearl-veterinary-partners/'],
 2: ['https://www.facebook.com/AEHGR'],
 3: ['https://www.facebook.com/Willow-Run-Veterinary-Services-190323027661850/']}

In [11]:
import pandas as pd
import time
from bs4 import BeautifulSoup
import requests
from fake_useragent import UserAgent

class WebScraper:

    def __init__(self, urls):
        self.data = None
        self.urls = list(urls)
        self.smm_dict = None

    def scrap_smm_info_from_pages(self):
        social_media_patterns = ['www.facebook.com', 'www.instagram.com', 'www.linkedin.com', 'www.twitter.com',
                                  'facebook.com', 'instagram.com', 'linkedin.com', 'twitter.com', 'x.com']
        ua = UserAgent()
        fake_headers = {'User-Agent': ua.chrome}
        smm_dict = {}

        for id4, smm_find in enumerate(self.urls):
            smm_links = set()
            try:
                response2 = requests.get(smm_find, timeout=8, headers=fake_headers, verify=False)
                soupy = BeautifulSoup(response2.text, 'html.parser')
                body = soupy.find('body')

                if body is not None:
                    href_values = [link.get('href') for link in body.find_all('a') if link.get('href')]

                    for href in href_values:
                        for pattern in social_media_patterns:
                            if pattern in href:
                                smm_links.add(href)

            except Exception as e:
                print(f"Error for URL {smm_find}: {e}")
                pass

            if smm_links:
                smm_dict[id4] = list(smm_links)

        self.smm_dict = smm_dict
        return smm_dict

# Example usage:
url_list = ["https://example1.com", "https://example2.com", "https://example3.com"]
scraper = WebScraper(url_list)

result = scraper.scrap_smm_info_from_pages()
print("Social Media Links:")
print(result)


c:\Users\Nick\anaconda3\envs\myenviorment\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'example1.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\Nick\anaconda3\envs\myenviorment\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'example2.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\Nick\anaconda3\envs\myenviorment\Lib\site-packages\urllib3\connectionpool.py:1045: InsecureRequestWarning: Unverified HTTPS request is being made to host 'example3.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(
c:\Users\Nick\an

Social Media Links:
{2: ['https://stickybudshfx.com/', '/domain/lnkwjx.com', 'https://lnkwjx.com']}


In [34]:
url = 'https://coitanimalclinic.com/'

first_file = Csv_scrapper(url)
first_file.clean_csv_file(url)

TypeError: Csv_scrapper.clean_csv_file() missing 1 required positional argument: 'csv_file'

In [ ]:
def create_df_cleaned_from_smm(smm_dict):  
    global df_of_smm_opt

    df_of_smm_opt = pd.DataFrame(smm_dict.items(), columns=['ID', 'Social Media Links'])
    
    platforms = ['Facebook', 'Instagram', 'LinkedIn','Twitter']

    for platform in platforms:
        df_of_smm_opt[platform] = df_of_smm_opt['Social Media Links'].apply(
            lambda links: [link for link in links if platform.lower() in link.lower()][0] if any(platform.lower() in link.lower() for link in links) else None
        )
    df_of_smm_opt = df_of_smm_opt.drop('Social Media Links', axis=1)
    
    return df_of_smm_opt

In [ ]:
if __name__ == '__main__':
    create_df_cleaned_from_smm(scrap_smm_info_from_pages(get_accounts_websites_for_smm_scrap()))

df_of_smm_opt

save = df_of_smm_opt.to_csv('smm_25k_homepage_scrapped.csv')